In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV,train_test_split
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor


In [3]:
df = pd.read_csv("./data/stud.csv")
X = df.drop(columns=["math_score"],axis=1)
y = df["math_score"]

In [4]:
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocesser = ColumnTransformer(
    [
        ("OneHotEncoder",oh_transformer,cat_features),
        ("StandardScaler",numeric_transformer,num_features)
     ]

)

In [5]:
X_transformed = preprocesser.fit_transform(X)

In [6]:
X_train,X_test,y_train,y_test = train_test_split(X_transformed,y,random_state=42)
X_train.shape,X_test.shape

((750, 19), (250, 19))

In [7]:
def evaluate_model(y,y_pred):
    mae = mean_absolute_error(y,y_pred)
    mse = mean_squared_error(y,y_pred)
    rmse = np.sqrt(mse)
    r2_square = r2_score(y,y_pred)
    return mae,mse,rmse,r2_square

In [8]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso":Lasso(),
    "Ridge":Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decission Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(),
    "CatBoost Regressor": CatBoostRegressor(),
    "AdaBoost Regressor": AdaBoostRegressor()
          }

model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae, model_train_mse, model_train_rmse, model_train_r2 = evaluate_model(y_train,y_train_pred)
    model_test_mae, model_test_mse, model_test_rmse, model_test_r2 = evaluate_model(y_test,y_test_pred)

    print (list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print("Model Performance for Training Set")
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print ("===================================")

    print("Model Performance for Testing Set")
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))

    r2_list.append(model_test_r2)
    print ("="*35)
    print("\n")






Linear Regression
Model Performance for Training Set
- Root Mean Squared Error: 5.3039
- Mean Absolute Error: 4.2493
- R2 Score: 0.8740
Model Performance for Testing Set
- Root Mean Squared Error: 5.4344
- Mean Absolute Error: 4.3092
- R2 Score: 0.8800


Lasso
Model Performance for Training Set
- Root Mean Squared Error: 6.5515
- Mean Absolute Error: 5.1837
- R2 Score: 0.8077
Model Performance for Testing Set
- Root Mean Squared Error: 6.6541
- Mean Absolute Error: 5.2217
- R2 Score: 0.8200


Ridge
Model Performance for Training Set
- Root Mean Squared Error: 5.2976
- Mean Absolute Error: 4.2368
- R2 Score: 0.8743
Model Performance for Testing Set
- Root Mean Squared Error: 5.4788
- Mean Absolute Error: 4.3354
- R2 Score: 0.8780


K-Neighbors Regressor
Model Performance for Training Set
- Root Mean Squared Error: 5.7920
- Mean Absolute Error: 4.5864
- R2 Score: 0.8497
Model Performance for Testing Set
- Root Mean Squared Error: 7.1327
- Mean Absolute Error: 5.5208
- R2 Score: 0.7932




In [12]:
models_r2_scores = pd.DataFrame(zip(model_list,r2_list),columns=["model_name","r2_score"]).sort_values("r2_score",ascending=False)